In [1]:
import torch
from torch import nn

from models.resnet import ResnetGenerator, ResnetBlock, InverseTanh

from tqdm import trange
from torch.optim import LBFGS

from itertools import product
import numpy as np
from skimage import data
import skimage
import matplotlib.pyplot as plot
import PIL.Image

In [2]:
class CustomModel(nn.Module):
    
    def __init__(self):
        super(CustomModel, self).__init__()
        self.model = nn.Sequential(InverseTanh(),
                                   nn.Conv2d(3, 40, 3, 2, 1),
                                   ResnetBlock(40, 'zero', nn.BatchNorm2d, False, False),
                                   nn.ConvTranspose2d(40, 3, 3, 2, 1),
                                   nn.Tanh())
        
    def forward(self, x):
        return self.model(x)

In [3]:
model = ResnetGenerator(3, 3)
# model = CustomModel()
model.eval();

In [4]:
def compute_kernel(n_input, n_output, kernel, stride, n_features):
    weights = torch.zeros([n_output, n_input, kernel[0], kernel[0]])
    center = kernel[0] // 2
    
    assert stride[0] == stride[1]
    assert kernel[0] == kernel[1]
    s = stride[0]
    for feature in range(n_features):
        for i, j in np.ndindex(stride):
            weights[feature*s*s + i*s + j, feature, center + i, center + j] = 1

    return weights

def compute_kernel_transpose(n_input, n_output, kernel, stride, n_features):
    weights = torch.zeros([n_input, n_output, kernel[0], kernel[0]])
    center = kernel[0] // 2
    
    assert stride[0] == stride[1]
    assert kernel[0] == kernel[1]
    assert stride[0] < kernel[0]
    
    s = stride[0]
    for feature in range(n_features//s//s):
        for i, j in np.ndindex(stride):
            weights[feature*s*s + i*s + j, feature,  center + i, center + j] = 1

    return weights

In [5]:
def zero_resblock(layer):
    layers = list(layer.conv_block.children())

    nn.init.normal(layers[0].weight, 0.00, 0.001)
    if layers[0].bias is not None: nn.init.constant(layers[0].bias, 0)
    nn.init.constant(layers[1].weight, 1)
    if layers[1].bias is not None: nn.init.constant(layers[1].bias, 0)
    nn.init.normal(layers[3].weight, 0.00, 0.001)
    if layers[3].bias is not None: nn.init.constant(layers[3].bias, 0)
    nn.init.constant(layers[4].weight, 1)
    nn.init.constant(layers[4].running_var, 1)
    nn.init.constant(layers[4].running_mean, 0)
    if layers[4].bias is not None: nn.init.constant(layers[4].bias, 0)

In [6]:
n_features_info = 3

for layer in model.model.children():
    if layer.__class__ == nn.BatchNorm2d:
        nn.init.zeros_(layer.bias)
        nn.init.ones_(layer.weight)
        nn.init.zeros_(layer.running_mean)
        nn.init.ones_(layer.running_var)
    elif layer.__class__ == ResnetBlock:
        zero_resblock(layer)
    elif layer.__class__ == nn.Conv2d:
        kernel = compute_kernel(layer.in_channels, layer.out_channels, 
                                layer.kernel_size, layer.stride, n_features_info)
        layer.weight.data = kernel
        if layer.bias is not None: nn.init.zeros_(layer.bias)
        n_features_info *= layer.stride[0] * layer.stride[1]
    elif layer.__class__ == nn.ConvTranspose2d:
        kernel = compute_kernel_transpose(layer.in_channels, layer.out_channels, 
                                layer.kernel_size, layer.stride, n_features_info)
        layer.weight.data = kernel
        if layer.bias is not None: nn.init.zeros_(layer.bias)
        n_features_info = n_features_info // (layer.stride[0] * layer.stride[1])
        
    print("{:<15} > {}".format(layer.__class__.__name__, n_features_info))

InverseTanh     > 3
Conv2d          > 3
BatchNorm2d     > 3
ReLU            > 3
Conv2d          > 12
BatchNorm2d     > 12
ReLU            > 12
Conv2d          > 48
BatchNorm2d     > 48
ReLU            > 48
ResnetBlock     > 48
ResnetBlock     > 48
ResnetBlock     > 48
ResnetBlock     > 48
ResnetBlock     > 48
ResnetBlock     > 48
ConvTranspose2d > 12
BatchNorm2d     > 12
ReLU            > 12
ConvTranspose2d > 3
BatchNorm2d     > 3
ReLU            > 3
Conv2d          > 3
Tanh            > 3


/home/daniilhayrapetyan/.virtualenvs/test/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  after removing the cwd from sys.path.
/home/daniilhayrapetyan/.virtualenvs/test/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  
/home/daniilhayrapetyan/.virtualenvs/test/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  import sys
/home/daniilhayrapetyan/.virtualenvs/test/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  
/home/daniilhayrapetyan/.virtualenvs/test/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  # Remove the CWD from sys.path while we load stuff.
/home/daniilhayrapetyan/

In [7]:
image = skimage.img_as_float(data.astronaut())

image_t = torch.Tensor(image.transpose(2, 0, 1))[None]

In [8]:
image_t1 = model.forward(image_t)
image_1 = image_t1.detach().numpy()[0].transpose(1, 2, 0)

In [9]:
# PIL.Image.fromarray(skimage.img_as_ubyte(image_1))

In [10]:
# PIL.Image.fromarray(skimage.img_as_ubyte(image))